In [103]:
import pandas as pd
from ast import literal_eval
from ru_sent_tokenize import ru_sent_tokenize as st
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
from pprint import pprint

In [62]:
df = pd.read_csv('additional_data/parsed_corpus_changed.csv')
df.target = df.target.apply(literal_eval)
df.source = df.source.apply(literal_eval)

In [63]:
df

,source,target,link
0,[В июне в этнографическом музее «Торум Маа» го...,[Лӯпта э̄тпост Ханты-Мансийск ӯст «То̄рум Ма...,https://khanty-yasang.ru//luima-seripos/no-14-...
1,"[Демид Чамалтынов, 9 лет, – Я живу в Ханты-Ман...","[Демид Чамалтынов, 9 та̄л, – «Та̄ксар ма̄хум» ...",https://khanty-yasang.ru//luima-seripos/no-16-...
2,[В Берёзовском районе в семи километрах ниже о...,[Ха̄льӯс районт Кульпас па̄вылныл ло̄ӈхаль са...,https://khanty-yasang.ru//luima-seripos/no-17-...
3,[Я хочу написать про своих детей. Этот год для...,[Ам амки на̄врама̄гум урыл хансуӈкв таӈхе̄гум....,https://khanty-yasang.ru//luima-seripos/no-13-...
4,[Каждое лето Ханты-Мансийский этнографический ...,[Ка̄сыӈ туи пора Ханты-Мансийск ӯст о̄лнэ на̄...,https://khanty-yasang.ru//luima-seripos/no-17-...
...,...,...,...
3746,[С хантыйской мастерицей Надеждой Алексеевной ...,[Надежда Алексеевна Гришкина ты та̄л атпан нуп...,https://khanty-yasang.ru//luima-seripos/no-24-...
3747,[Для жителей сельского поселения села Саранпау...,[Ма̄н округувт са̄в са̄лыт туп тит ма̄т о̄ньща...,https://khanty-yasang.ru//luima-seripos/no-5-1...
3748,[Альбина Волосовская уже много лет живет и раб...,[Ты хурит Октябрьский па̄вылт о̄лнэ ма̄ньщи нэ...,https://khanty-yasang.ru//luima-seripos/no-14-...
3749,[В Ханты-Мансийске в июне месяце состоялись «I...,[Ха̄льӯст ма̄ньлат ма̄ньщи хум Михаил Яркин о...,https://khanty-yasang.ru//luima-seripos/no-13-...


In [176]:
for paragraph in [st(el) for el in df.target.loc[230]]:
    for sent in paragraph[:2]:
        print(sent, end='\n--------------------\n')

TypeError: expected string or bytes-like object

In [45]:
model = SentenceTransformer('sentence-transformers/LaBSE', device='cuda')# ('sergeyzh/rubert-tiny-turbo')

C:\Users\1645286\venvs\py310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [65]:
df.source = df.source.apply(lambda x: [st(el) for el in x])
df.target = df.target.apply(lambda x: [st(el) for el in x])

In [70]:
# удаление пустых абзацев
df.target = df.target.apply(lambda x: [el for el in x if len(el) > 0])
df.source = df.source.apply(lambda x: [el for el in x if len(el) > 0])

In [71]:
# удаление пустых предложений
df.target = df.target.apply(lambda x: [[c for c in el if len(c) > 0] for el in x])
df.source = df.source.apply(lambda x: [[c for c in el if len(c) > 0] for el in x])

In [80]:
i = 4
for par_mansi, par_rus in zip(df.target.loc[i], df.source.loc[i]):
    print(par_mansi, '\nPEREVOD:\n', par_rus, end='\n---------------\n')

['Ка̄сыӈ туи пора Ханты-Мансийск ӯст о̄лнэ на̄врамыт магыс «То̄рум Маа» музеит ханищтап ва̄раве, ма̄ньщи ла̄тӈыл ла̄ваве – «Та̄ксар ма̄хум».', 'Тув ялантам на̄врамыт тох потыртасыт.'] 
PEREVOD:
 ['Каждое лето Ханты-Мансийский этнографический музей под открытым небом «Торум Маа» для маленьких посетителей проводит детские смены «Таксар Махум» - «Крепкие люди», где ребят учат навыкам выживания в лесу.', 'Своими впечатлениями от посещения смены поделилась юная жительница Ханты-Мансийска Александра Заболотская:']
---------------
['Александра Заболотская, 13 та̄л'] 
PEREVOD:
 ['– Мне 13 лет, учусь в 5 школе, перешла в 8 класс.', 'Навыки эти полезные, научили разжигать огонь разными способами, ставить палатку, и это обязательно надо знать, вдруг ты заблудишься в лесу.', 'Научилась ставить разные ловушки.', 'Очень важно, что нас знакомят с традиционным укладом жизни коренных народов Севера.']
---------------
['– А̄нумн хӯрумхойплов та̄л, 5 школат 8 классыт ханищтахтэ̄гум.', 'Ханищтап – сака 

In [ ]:
# first row - X[0] with all Y
    # second row - X[1] with all Y
    # first column - Y[0] with all X
    # second column - Y[1] with all X

In [151]:
temp = df.loc[1:1]
for _, row in tqdm(temp.iterrows(), total=len(temp)):
    mansi_sents = sum(row.target, [])
    rus_sents = sum(row.source, [])
    mansi_embs = model.encode(mansi_sents)
    rus_embs = model.encode(rus_sents)
    cos_sim = cosine_similarity(rus_embs, mansi_embs) * 100

    rus_top5_mean = np.mean(np.sort(cos_sim, axis=1)[:, -5:], axis=1)
    mansi_top5_mean = np.mean(np.sort(cos_sim, axis=0)[-5:, :], axis=0)

    cos_sim_norm_rows = cos_sim / rus_top5_mean[:, np.newaxis]
    cos_sim_norm = cos_sim_norm_rows / mansi_top5_mean[np.newaxis, :]
    
    rus_to_mansi_indices = np.argmax(cos_sim, axis=1)
    mansi_to_rus_indices = np.argmax(cos_sim, axis=0)
    
    rus_to_mansi = [
        {
            'id_rus': i,
            'id_mansi': idx,
            'rus': rus_sents[i],
            'mansi': mansi_sents[idx],
            'score': cos_sim[i, idx]
        }
        for i, idx in enumerate(rus_to_mansi_indices)
    ]
    rus_to_mansi = sorted(rus_to_mansi, key=lambda x: x['id_rus'])
    
    mansi_to_rus = [
        {
            'id_rus': idx,
            'id_mansi': i,
            'mansi': mansi_sents[i],
            'rus': rus_sents[idx],
            'score': cos_sim[idx, i]
        }
        for i, idx in enumerate(mansi_to_rus_indices)
    ]
    mansi_to_rus = sorted(mansi_to_rus, key=lambda x: x['id_mansi'])

  0%|          | 0/1 [00:00<?, ?it/s]

In [178]:
cos_sim.shape

(17, 16)

In [188]:
pd.merge(
    pd.DataFrame(rus_to_mansi).rename(
        columns={
            'rus': 'rus_from_rus',
            'mansi': 'mansi_from_rus',
            'id_mansi': 'id_mansi_from_rus',
            'score': 'score_from_rus'
        }
    ),
    pd.DataFrame(mansi_to_rus).rename(
        columns={
            'rus': 'rus_from_mansi',
            'mansi': 'mansi_from_mansi',
            'id_mansi': 'id_mansi_from_mansi',
            'score': 'score_from_mansi'
        }
    ),
    on='id_rus',
    how='left'
) \
    .groupby('id_rus', as_index=False) \
    .agg(
        id_rus=('id_rus', 'max'),
        # rus
        id_mansi_from_rus=('id_mansi_from_rus', 'max'),
        rus_from_rus=('rus_from_rus', 'max'),
        mansi_from_rus=('mansi_from_rus', 'max'),
        score_from_rus=('score_from_rus', 'max'),
        # mansi
        id_mansi_from_mansi=('id_mansi_from_mansi', lambda x: [int(el) for el in x if pd.notna(el)] if x.notna().sum() > 0 else None),
        rus_from_mansi=('rus_from_mansi', lambda x: [el for el in x if pd.notna(el)] if x.notna().sum() > 0 else None),
        mansi_from_mansi=('mansi_from_mansi', lambda x: [el for el in x if pd.notna(el)] if x.notna().sum() > 0 else None),
        score_from_mansi=('score_from_mansi', lambda x: [el for el in x if pd.notna(el)] if x.notna().sum() > 0 else None)
    )

,id_rus,id_mansi_from_rus,rus_from_rus,mansi_from_rus,score_from_rus,id_mansi_from_mansi,rus_from_mansi,mansi_from_mansi,score_from_mansi
0,0,0,"Демид Чамалтынов, 9 лет","Демид Чамалтынов, 9 та̄л",95.432877,[0],"[Демид Чамалтынов, 9 лет]","[Демид Чамалтынов, 9 та̄л]",[95.43287658691406]
1,1,5,"– Я живу в Ханты-Мансийске, обучаюсь в школе № 2.","Ам Ханты-Мансийск ӯст о̄лэ̄гум, 2 школа ханищтахтэ̄гум.",72.602448,"[4, 5, 11]","[– Я живу в Ханты-Мансийске, обучаюсь в школе № 2., – Я живу в Ханты-Мансийске, обучаюсь в школе № 2., – Я живу в Ханты-Мансийске, обучаюсь в школе № 2.]","[Тыт сака пӯмащ о̄лыс, а̄гит-пыгыт ёт ёнгысӯв., Ам Ханты-Мансийск ӯст о̄лэ̄гум, 2 школа ханищтахтэ̄гум., Ам Ханты-Мансийск ӯс колледжит китыт курсыт ханищтахтэ̄гум.]","[9.710014343261719, 72.60244750976562, 60.135066986083984]"
2,2,8,"В этом году я впервые посетил интерактивную программу «Крепкие люди» и очень благодарен маме, что она записала меня на замечательную смену, где мы с пользой проводим время на свежем воздухе и узнаём много нового о том, как жили коренные малочисленные народы Севера.","Анастасия Тимофеева, 17 та̄л",27.651772,None,None,None,None
3,3,3,"Наш наставник, Вячеслав Юрьевич, нам всё рассказывают, и мы на практике сами учимся этим навыкам, изготавливаем ловушки на диких зверей.","Вячеслав Юрьевич ат хо̄тал сыс ма̄нав ханищтастэ, хумыс э̄рнэ щирыл алыщлапыт тӯщтуӈкве.",54.424183,"[3, 6]","[Наш наставник, Вячеслав Юрьевич, нам всё рассказывают, и мы на практике сами учимся этим навыкам, изготавливаем ловушки на диких зверей., Наш наставник, Вячеслав Юрьевич, нам всё рассказывают, и мы на практике сами учимся этим навыкам, изготавливаем ловушки на диких зверей.]","[Вячеслав Юрьевич ат хо̄тал сыс ма̄нав ханищтастэ, хумыс э̄рнэ щирыл алыщлапыт тӯщтуӈкве., Ще̄ранка та̄л на̄й пе̄ламтаӈкве о̄с ха̄сэ̄гум, а̄нумн ква̄лыг лё̄ӈх хосыт ё̄муӈкве пӯмыщ о̄лыс, лё̄мвой тэ̄рпи ма̄нти ва̄рсӯв.]","[54.4241828918457, 14.480720520019531]"
4,4,8,"В лагере очень интересно, за эти дни у меня появилось много новых друзей.","Анастасия Тимофеева, 17 та̄л",31.100285,[10],"[В лагере очень интересно, за эти дни у меня появилось много новых друзей.]","[Тав са̄в ёмас ла̄тыӈ ты лагерь урыл потыртас, ам о̄с тыг ёхтуӈкве тахмаясум.]",[28.564388275146484]
5,5,5,"Рад, что научился разжигать костёр, а также мне очень понравился верёвочный парк, где мы с ребятами испытали себя со специальным страховочным снаряжением на трассе с препятствиями на выносливость.","Ам Ханты-Мансийск ӯст о̄лэ̄гум, 2 школа ханищтахтэ̄гум.",26.666311,None,None,None,None
6,6,15,"Мы играли в традиционные игры обско-угорских народов, готовили натуральные средства для защиты от комаров и мошек.","Ам амп о̄ньще̄гум, тав наме Урма, тав немецкий овчарка, тав ётэ «ЗооЗож» суссылтапт о̄лсум, са̄всыр мир э̄лы-па̄лт кит о̄выл места тот висум, о̄йттур э̄тпост «ПикникХМ-2023» ма̄н ялсӯв, тав мо̄т ампыт ёт тот касыс.",25.541231,None,None,None,None
7,7,1,В следующем году я вновь вернусь на смену «Таксар махум» – «Крепкие люди».,– «Та̄ксар ма̄хум» – «Крепкие люди» сменан ам ты та̄л о̄выл щёс ёхтысум.,57.511723,[1],[В следующем году я вновь вернусь на смену «Таксар махум» – «Крепкие люди».],[– «Та̄ксар ма̄хум» – «Крепкие люди» сменан ам ты та̄л о̄выл щёс ёхтысум.],[57.511722564697266]
8,8,11,"Я очень благодарен организаторам проведения интерактивной программы за то, что знакомят нас с такими народами как ханты, манси и ненцы, с их традиционным жизненным укладом.",Ам Ханты-Мансийск ӯс колледжит китыт курсыт ханищтахтэ̄гум.,31.351004,None,None,None,None
9,9,8,"Анастасия Тимофеева, 17 лет","Анастасия Тимофеева, 17 та̄л",95.140656,[8],"[Анастасия Тимофеева, 17 лет]","[Анастасия Тимофеева, 17 та̄л]",[95.14065551757812]


In [179]:
pd.set_option('display.max_colwidth', None)

In [187]:
pd.DataFrame(rus_to_mansi)#.query('score > 50')

,id_rus,id_mansi,rus,mansi,score
0,0,0,"Демид Чамалтынов, 9 лет","Демид Чамалтынов, 9 та̄л",95.432877
1,1,5,"– Я живу в Ханты-Мансийске, обучаюсь в школе № 2.","Ам Ханты-Мансийск ӯст о̄лэ̄гум, 2 школа ханищтахтэ̄гум.",72.602448
2,2,8,"В этом году я впервые посетил интерактивную программу «Крепкие люди» и очень благодарен маме, что она записала меня на замечательную смену, где мы с пользой проводим время на свежем воздухе и узнаём много нового о том, как жили коренные малочисленные народы Севера.","Анастасия Тимофеева, 17 та̄л",27.651772
3,3,3,"Наш наставник, Вячеслав Юрьевич, нам всё рассказывают, и мы на практике сами учимся этим навыкам, изготавливаем ловушки на диких зверей.","Вячеслав Юрьевич ат хо̄тал сыс ма̄нав ханищтастэ, хумыс э̄рнэ щирыл алыщлапыт тӯщтуӈкве.",54.424183
4,4,8,"В лагере очень интересно, за эти дни у меня появилось много новых друзей.","Анастасия Тимофеева, 17 та̄л",31.100285
5,5,5,"Рад, что научился разжигать костёр, а также мне очень понравился верёвочный парк, где мы с ребятами испытали себя со специальным страховочным снаряжением на трассе с препятствиями на выносливость.","Ам Ханты-Мансийск ӯст о̄лэ̄гум, 2 школа ханищтахтэ̄гум.",26.666311
6,6,15,"Мы играли в традиционные игры обско-угорских народов, готовили натуральные средства для защиты от комаров и мошек.","Ам амп о̄ньще̄гум, тав наме Урма, тав немецкий овчарка, тав ётэ «ЗооЗож» суссылтапт о̄лсум, са̄всыр мир э̄лы-па̄лт кит о̄выл места тот висум, о̄йттур э̄тпост «ПикникХМ-2023» ма̄н ялсӯв, тав мо̄т ампыт ёт тот касыс.",25.541231
7,7,1,В следующем году я вновь вернусь на смену «Таксар махум» – «Крепкие люди».,– «Та̄ксар ма̄хум» – «Крепкие люди» сменан ам ты та̄л о̄выл щёс ёхтысум.,57.511723
8,8,11,"Я очень благодарен организаторам проведения интерактивной программы за то, что знакомят нас с такими народами как ханты, манси и ненцы, с их традиционным жизненным укладом.",Ам Ханты-Мансийск ӯс колледжит китыт курсыт ханищтахтэ̄гум.,31.351004
9,9,8,"Анастасия Тимофеева, 17 лет","Анастасия Тимофеева, 17 та̄л",95.140656


In [186]:
pd.DataFrame(mansi_to_rus)#.query('score > 50')

,id_rus,id_mansi,mansi,rus,score
0,0,0,"Демид Чамалтынов, 9 та̄л","Демид Чамалтынов, 9 лет",95.432877
1,7,1,– «Та̄ксар ма̄хум» – «Крепкие люди» сменан ам ты та̄л о̄выл щёс ёхтысум.,В следующем году я вновь вернусь на смену «Таксар махум» – «Крепкие люди».,57.511723
2,13,2,"Татем та ёмас, омам а̄нум ты сменан хансыстэ, на̄врамыт ма̄гыс музейт рӯпитан хо̄тпат са̄всыр ханищтапыт ва̄рыгласыт.","Конечно, хотелось бы отметить верёвочный парк, там нас обучали, как в экстремальных условиях пройти препятствия, преодолевая свой страх.",17.164192
3,3,3,"Вячеслав Юрьевич ат хо̄тал сыс ма̄нав ханищтастэ, хумыс э̄рнэ щирыл алыщлапыт тӯщтуӈкве.","Наш наставник, Вячеслав Юрьевич, нам всё рассказывают, и мы на практике сами учимся этим навыкам, изготавливаем ловушки на диких зверей.",54.424183
4,1,4,"Тыт сака пӯмащ о̄лыс, а̄гит-пыгыт ёт ёнгысӯв.","– Я живу в Ханты-Мансийске, обучаюсь в школе № 2.",9.710014
5,1,5,"Ам Ханты-Мансийск ӯст о̄лэ̄гум, 2 школа ханищтахтэ̄гум.","– Я живу в Ханты-Мансийске, обучаюсь в школе № 2.",72.602448
6,3,6,"Ще̄ранка та̄л на̄й пе̄ламтаӈкве о̄с ха̄сэ̄гум, а̄нумн ква̄лыг лё̄ӈх хосыт ё̄муӈкве пӯмыщ о̄лыс, лё̄мвой тэ̄рпи ма̄нти ва̄рсӯв.","Наш наставник, Вячеслав Юрьевич, нам всё рассказывают, и мы на практике сами учимся этим навыкам, изготавливаем ловушки на диких зверей.",14.480721
7,11,7,"Яныг пӯмащипа «То̄рум Маа» музейт рӯпитан хо̄тпатн ла̄ве̄гум, ка̄сыӈ та̄л са̄в на̄врам тув вос ёхталы.","На летнюю смену «Таксар махум» – «Крепкие люди» меня привела моя подруга, Саша Заболотская.",27.897198
8,9,8,"Анастасия Тимофеева, 17 та̄л","Анастасия Тимофеева, 17 лет",95.140656
9,11,9,– Туи сменан «Та̄ксар ма̄хум» ам юрт-а̄гим Саша Заболотская ёт ёхтысум.,"На летнюю смену «Таксар махум» – «Крепкие люди» меня привела моя подруга, Саша Заболотская.",71.846680
